In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt


--2023-10-22 14:16:17--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.03s   

2023-10-22 14:16:17 (6.26 MB/s) - ‘names.txt’ saved [228145/228145]



In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)} # Map characters to integers
stoi['.'] = 0  # Assign integer 0 to the period character
itos = {i:s for s,i in stoi.items()}  # Reverse mapping of integers to characters
vocab_size = len(itos)  # Size of the vocabulary
print(itos)
print(vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
block_size = 3  # Context length: how many characters to predict the next one?


In [8]:
def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]  # Map character to its integer representation
      X.append(context)  # Add the context to X
      Y.append(ix)  # Add the target character to Y
      context = context[1:] + [ix]  # Shift context and append the new character

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y


In [9]:
# Shuffle and split the dataset into training, development, and test sets
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [10]:
n_embd = 10  # The dimensionality of the character embedding vectors
n_hidden = 64  # The number of neurons in the hidden layer of the MLP

In [11]:
g = torch.Generator().manual_seed(2147483647)  # For reproducibility
C  = torch.randn((vocab_size, n_embd), generator=g)  # Character embeddings
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)  # Layer 1 weights
b1 = torch.randn(n_hidden, generator=g) * 0.1  # Layer 1 biases
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1  # Layer 2 weights
b2 = torch.randn(vocab_size, generator=g) * 0.1  # Layer 2 biases


In [12]:
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0  # BatchNorm gain
bnbias = torch.randn((1, n_hidden))*0.1  # BatchNorm bias


In [13]:
# Define the network parameters as a list
parameters = [C, W1, b1, W2, b2, bngain, bnbias]

In [14]:
# Calculate the total number of parameters
print(sum(p.nelement() for p in parameters))

4137


In [15]:
# Enable gradients for the network parameters
for p in parameters:
  p.requires_grad = True

In [16]:
# Set batch size and create a minibatch
batch_size = 32
n = batch_size  # A shorter variable for convenience
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]  # Batch X, Y



In [17]:
# Forward pass: "chunkated" into smaller steps that are possible to backward one at a time
emb = C[Xb]  # Embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # Concatenate the vectors
hprebn = embcat @ W1 + b1  # Hidden layer pre-activation

In [18]:
# BatchNorm layer
# Calculate the mean and variance over the batch
bnmean = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmean
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5  # Add epsilon for stability
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact)  # Hidden layer
logits = h @ W2 + b2  # Output layer

In [19]:
# Cross-entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # Subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1  # If we use (1.0 / counts_sum) instead, backprop won't be bit exact
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()


In [20]:
# PyTorch backward pass
for p in parameters:
  p.grad = None
loss.backward()
